In [65]:
import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import scipy.optimize as optimize
import random

## Define the objective function

In [2]:
def mutual_potential(x1,x2):
    return sc.elementary_charge**2/(4*np.pi*sc.epsilon_0)*1/np.linalg.norm(x1-x2)

def array_patition(x,p):
    l = int(len(x)/p)
    output_array = np.array([[x[p*i+j] for j in np.arange(p)] for i in np.arange(l)])
    return output_array

In [60]:
def CoulombPotential(x,args):
    position_vec = array_patition(x,3)
    ion_number = len(position_vec)
    potential_vec = np.array([])
    for i in np.arange(ion_number):
        for j in np.arange(i):
            current_potential = mutual_potential(position_vec[i],position_vec[j])
            potential_vec = np.append(potential_vec,current_potential)
    return sum(potential_vec)

def HamonicPotential(x,args):
    position_vec = np.transpose(array_patition(x,3))
    omega = args['frequency']
    potential_vec = np.array([1/2*args['mass']*(omega[i]**2)*(position_vec[i]**2) for i in np.arange(3)])
    return sum([sum(potential_vec[i]) for i in np.arange(3)])

def CrystalPotential(x,args):
    return (CoulombPotential(x,args) + HamonicPotential(x,args))*10**22

## Find the Minimizer

In [106]:
mass_Yb = 70*sc.m_p+101*sc.m_n
trap_frequencies = 2*np.pi*np.array([0.41*10**6,0.37*10**6,1.4*10**6])
ionnumber = 10
x0=np.array([10**(-9+random.random())*i for i in np.arange(3*ionnumber)])
fprime = lambda x,args: optimize.approx_fprime(x, CrystalPotential, 0.01, {'mass':mass_Yb,'frequency':trap_frequencies})
crystal_geometry = minimize(CrystalPotential, x0, method='Powell', args={'mass':mass_Yb,'frequency':trap_frequencies})
print(crystal_geometry['x']-x0)
print("\nObjective function:")
print(CrystalPotential(crystal_geometry['x'],args={'mass':mass_Yb,'frequency':trap_frequencies}))

[-1.78656787e-06 -9.60212592e-06 -2.15276201e-09  4.84710357e-06
 -7.92762911e-06 -3.92821651e-08  2.10634503e-06  9.55100673e-06
 -2.09032553e-08 -4.63519065e-06  8.03799455e-06 -3.53710831e-08
  6.94375568e-06 -1.82341168e-06 -4.64858661e-08  1.24531032e-07
  2.72828369e-06 -2.70124176e-08 -7.05061923e-06 -4.69747660e-06
 -7.56138446e-08 -7.20715436e-06  1.76627137e-06 -5.61444869e-08
  6.77206645e-06  4.56055828e-06 -5.69497508e-08 -3.85336600e-07
 -3.13877746e-06 -1.10232204e-07]

Objective function:
15.981069696406985


In [107]:
np.savetxt("/Users/mu_qiao/Documents/OneDrive/Jupyter/Ion crystal geometry solver/Crystal_Geometry.txt",crystal_geometry['x'])
array_patition(crystal_geometry['x'],3)

array([[-1.78656787e-06, -9.59846695e-06, -4.26082799e-12],
       [ 4.85119615e-06, -7.91519916e-06, -3.64680198e-12],
       [ 2.15946029e-06,  9.56135507e-06, -1.41987922e-12],
       [-4.62473191e-06,  8.05460073e-06, -1.35148228e-12],
       [ 7.03543709e-06, -1.78715879e-06, -5.98004715e-12],
       [ 1.96150433e-07,  2.86148022e-06,  1.88435145e-12],
       [-7.03132623e-06, -4.67473332e-06,  1.01055209e-13],
       [-7.18412877e-06,  1.89041048e-06, -1.11278722e-12],
       [ 6.96593850e-06,  4.61696335e-06, -1.46681739e-12],
       [-3.42681653e-07, -2.94759576e-06,  8.23773256e-13]])